In [67]:
import sqlite3

In [68]:
conn = sqlite3.connect('./Dumps/lab2.db')

cur = conn.cursor()

PRESS (NAME, ADDR, PHNO) 

BOOK_INFO (B_ID, TITLE, PRESS_NAME, YEAR_OF_PUBLICATION) 

AUTHOR_INFO (B_ID, TITLE,AUTHOR_NAME) 

DEPT_LIB(DEPT_NO, DEPT_NAME,DEPT_BLOCK) 

NO_OF_COPIES(B_ID, DEPT_NAME,TOTAL_COPIES) 

WITHDRAW_BOOK(B_ID,DEPT_NAME,LIB_ID,WITHDRAW_DATE,RETURN _DATE) 

LIBRARYID(LIB_ID) 

In [69]:
createTables = '''
BEGIN;

CREATE TABLE PRESS (
    NAME TEXT PRIMARY KEY,
    ADDR TEXT,
    PHNO TEXT
);


CREATE TABLE BOOK_INFO (
    B_ID TEXT PRIMARY KEY,
    TITLE TEXT PRIMAY KEY,
    PRESS_NAME TEXT REFERENCES PRESS(NAME) ON UPDATE CASCADE,
    YEAR_OF_PUBLICATION INT
);

CREATE TABLE AUTHOR_INFO (
    B_ID TEXT PRIMARY KEY,
    TITLE TEXT REFERENCES BOOK_INFO(TITLE) ON UPDATE CASCADE,
    AUTHOR_NAME TEXT
);

CREATE TABLE DEPT_LIB (
    DEPT_NO INT PRIMARY KEY,
    DEPT_NAME TEXT,
    DEPT_BLOCK TEXT
);

CREATE TABLE NO_OF_COPIES (
    B_ID TEXT PRIMARY KEY,
    DEPT_NO INT REFERENCES DEPT_LIB(DEPT_NO) ON UPDATE CASCADE,
    TOTAL_COPIES INT
);

CREATE TABLE LIBRARYID (
    LIB_ID INT PRIMARY KEY
);

CREATE TABLE WITHDRAW_BOOK (
    B_ID TEXT PRIMARY KEY,
    DEPT_NO TEXT REFERENCES DEPT_LIB(DEPT_NO) ON UPDATE CASCADE,
    LIB_ID INT REFERENCES LIBRARYID(LIB_ID) ON UPDATE CASCADE,
    WITHDRAW_DATE DATE,
    RETURN_DATE DATE 
);

COMMIT;
'''

In [70]:
cur.executescript(createTables)

In [71]:
press = [
    ('holmes', '221B', '132324'),
    ('holmes Jr.', 'Pall Mall', '156324')
]

cur.executemany('INSERT INTO PRESS VALUES (?, ?, ?)', press)

In [72]:
book_info = [
    ('b1', 'On the Origins of Tree Worship', 'holmes', '1896'),
    ('b2', 'Bee Culture', 'holmes Jr.', '1890')
]

cur.executemany('INSERT INTO BOOK_INFO VALUES (?, ?, ?, ?)', book_info)

In [73]:
author_info = [
    ('b1', 'On the Origins of Tree Worship', 'Sigerson'),
    ('b2', 'Bee Culture', 'Sherlock')
]

cur.executemany('INSERT INTO AUTHOR_INFO VALUES (?, ?, ?)', author_info)

In [74]:
dept_lib = [
    (1, 'D1', 'Block 1'),
    (2, 'D2', 'Block 2')
]

cur.executemany('INSERT INTO DEPT_LIB VALUES (?, ?, ?)', dept_lib)

In [75]:
no_of_copies = [
    ('b1', 1, 69),
    ('b2', 2, 420)
]

cur.executemany('INSERT INTO NO_OF_COPIES VALUES (?, ?, ?)', no_of_copies)

In [79]:
libraryid = [
    (1, ),
    (2, )
]

cur.executemany('INSERT INTO LIBRARYID VALUES (?)', libraryid)

In [81]:
withdraw_book = [
    ('b1', 1, 1, '2022-08-05', '2022-08-09'),
    ('b2', 2, 2, '2022-07-09', '2022-08-11')
]

cur.executemany('INSERT INTO WITHDRAW_BOOK VALUES (?, ?, ?, ?, ?)', withdraw_book)

In [82]:
conn.commit()

DISPLAY THE FOLLOWING DETAILS OF BOOKS FROM THE LIBRARY USING A SINGLE QUERY [B_ID, TITLE, PRESS_NAME, AUTHOR_NAME, TOTAL_COPIES]

In [86]:
res1 = cur.execute('''
    SELECT B.B_ID, B.TITLE, B.PRESS_NAME, A.AUTHOR_NAME, N.TOTAL_COPIES
    FROM BOOK_INFO B 
    INNER JOIN AUTHOR_INFO A ON A.B_ID = B.B_ID
    INNER JOIN NO_OF_COPIES N ON B.B_ID = N.B_ID
''')

res1.fetchall()

[('b1', 'On the Origins of Tree Worship', 'holmes', 'Sigerson', 69),
 ('b2', 'Bee Culture', 'holmes Jr.', 'Sherlock', 420)]

Display the details of the students withdrawn the books between 16.05.2022 and 16.08.2022 




Drop a book in BOOK_INFO table. Proof that the deletion is updated in other relevant tables. 

Partition the BOOK_INFO table based on the NO_OF_COPIES. Illustrate this with a simple query in Data Manipulation. 

In [65]:
conn.close()

In [85]:
import os

os.remove('./Dumps/lab2.db')

FileNotFoundError: [WinError 2] The system cannot find the file specified: './Dumps/lab3.db'